In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import gseapy as gp
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand

p = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(p)

from proc_revisions.utils import *
from proc_revisions.plotting import *
# from proc_revisions.mane_utils import *

In [2]:
od = '../../proc_revisions/'
config_file = f'{od}/config.yml'
with open(config_file) as f:
    config = yaml.safe_load(f)

In [3]:
ab = od+expand(config['lr']['talon']['fusion_fix']['ab'], species='human')[0]
filt_ab = od+expand(config['lr']['cerberus']['filt_ab'], species='human')[0]
read_annot = od+expand(config['lr']['talon']['full_annot'], species='human')[0]
t_metadata = od+expand(config['ref']['cerberus']['new_gtf_t_info'], species='human')[0]
lib_meta = od+expand(config['lr']['meta'], species='human')[0]
swan_file = od+expand(config['lr']['swan']['sg'], species='human')[0]
cerberus_h5 = od+expand(config['lr']['cerberus']['ca_triplets'], species='human', obs_col='sample')[0]
cerb_t_metadata = od+expand(config['lr']['cerberus']['gtf_t_info'], species='human')[0]
major_isos = od+expand(config['lr']['analysis']['major_isos'], species='human', obs_col='sample')[0]
pi_tpm_table = od+expand(config['lr']['mane']['pi_tpm']['triplet'], species='human', obs_col='sample')[0]

ref_t_metadata = od+expand(config['ref']['new_gtf_t_info'], species='human')[0]
ref_g_metadata = od+expand(config['ref']['new_gtf_g_info'], species='human')[0]
read_annot = od+expand(config['lr']['talon']['full_annot'], species='human')[0]

talon_ref_gtf = od+expand(config['ref']['talon']['gtf'], species='human')[0]
# pp_summary = '../'+expand(config['data']['p_pred']['summary'], species='human')[0]

ver = 'v40_cerberus'
min_tpm = 1
gene_subset = 'polya'
obs_col = 'sample'
go_gene_subset = 'protein_coding'
predom_iso_subset = 'protein_coding'

m_lib_meta = od+expand(config['lr']['meta'], species='mouse')[0]

In [6]:
trip_summ = od+expand(config['lr']['param_search']['cerberus']['trip_summary'], species='human')[0]

In [19]:
df = pd.read_csv(trip_summ, sep='\t', nrows=20000)

In [20]:
cols = ['source', 'gid', 'n_tss', 'n_tes', 'n_ic', 'n_iso', 'splicing_ratio',
       'tss_ratio', 'tes_ratio', 'spl_ratio', 'sector', 'gname', 'sample',
       'gene_tpm', 'file', 'tss_dist', 'tes_dist',
        'tss_slack', 'tes_slack',
        'tss_agg_dist', 'tes_agg_dist']
df.columns = cols

In [24]:
df['source'] = df['source']+'_'+\
               df['tss_dist'].astype(str)+'_'+\
               df['tes_dist'].astype(str)+'_'+\
               df['tss_slack'].astype(str)+'_'+\
               df['tes_slack'].astype(str)+'_'+\
               df['tss_agg_dist'].astype(str)+'_'+\
               df['tes_agg_dist'].astype(str)


In [22]:
ca = cerberus.read(cerberus_h5)

In [26]:
df.head()

,source,gid,n_tss,n_tes,n_ic,n_iso,splicing_ratio,tss_ratio,tes_ratio,spl_ratio,...,gname,sample,gene_tpm,file,tss_dist,tes_dist,tss_slack,tes_slack,tss_agg_dist,tes_agg_dist
0,obs_det_0_0_0_0_0_0,ENSG00000000003,3.0,11.0,9.0,34.0,1.285714,0.196262,0.719626,0.084112,...,TSPAN6,NaN,NaN,data/human/lr/param_search/0_0_0_0_0_0/sample_...,0,0,0,0,0,0
1,obs_det_0_0_0_0_0_0,ENSG00000000005,1.0,1.0,1.0,1.0,1.000000,0.333333,0.333333,0.333333,...,TNMD,NaN,NaN,data/human/lr/param_search/0_0_0_0_0_0/sample_...,0,0,0,0,0,0
2,obs_det_0_0_0_0_0_0,ENSG00000000419,6.0,5.0,8.0,15.0,1.454545,0.481752,0.401460,0.116788,...,DPM1,NaN,NaN,data/human/lr/param_search/0_0_0_0_0_0/sample_...,0,0,0,0,0,0
3,obs_det_0_0_0_0_0_0,ENSG00000000457,5.0,6.0,5.0,10.0,0.909091,0.419847,0.503817,0.076336,...,SCYL3,NaN,NaN,data/human/lr/param_search/0_0_0_0_0_0/sample_...,0,0,0,0,0,0
4,obs_det_0_0_0_0_0_0,ENSG00000000460,6.0,5.0,6.0,19.0,1.090909,0.496241,0.413534,0.090226,...,C1orf112,NaN,NaN,data/human/lr/param_search/0_0_0_0_0_0/sample_...,0,0,0,0,0,0
